# LSTM

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

#import libraries
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np

# Import Statsmodels
from statsmodels.tsa.stattools import adfuller

In [48]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, validation_curve
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [49]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session 
tf.compat.v1.ConfigProto()

In [50]:
data = pd.read_csv('df5.csv')



array([133, 20.04, 52, 17, 96, 165, 13.7954,
       "['Positive', ' Neutral', ' Neutral', ' Positive', ' Positive', ' Positive', ' Positive', ' Positive', ' Neutral', ' Positive', ' Positive', ' Positive', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Positive', ' Positive', ' Positive', ' Neutral', ' Positive', ' Neutral', ' Neutral', ' Neutral', ' Positive', ' Positive', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Positive', ' Neutral', ' Neutral', ' Negative', ' Negative', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Positive', ' Neutral', ' Positive', ' Positive', ' Neutral', ' Positive', ' Positive', ' Negative', ' Positive', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Neutral', ' Positive', ' Positive', ' Neutral', ' Positive', ' Neutral', ' Positive', ' Neutral', ' Positive', ' Neutral', ' Neutral', ' Positive', ' Positive', ' Negative', ' Neutral', ' Positive', ' Positive', ' Positive', ' Neutral', ' Neutral', ' Positive', ' Neutral'

In [54]:
data['target'] = data.CLOSE.shift(-1)
data.loc[:,'direction'] = [0 if d1>=d2 else 1 for d1,d2 in zip(data.CLOSE, data.target)]
data = data.dropna()
data_values = data.values
data.info()
data.tail()

<class 'pandas.core.frame.DataFrame'>
Index: 2627 entries, 0 to 2626
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2627 non-null   int64  
 1   CLOSE              2627 non-null   float64
 2   Positive           2627 non-null   int64  
 3   Negative           2627 non-null   int64  
 4   Neutral            2627 non-null   int64  
 5   Total              2627 non-null   int64  
 6   summed_vader       2627 non-null   float64
 7   sentiments         2627 non-null   object 
 8   headlines          2627 non-null   object 
 9   stocks             2627 non-null   object 
 10  date               2627 non-null   object 
 11  dayOfWeek          2627 non-null   object 
 12  ave_vader          2627 non-null   float64
 13  daily_sentiment    2627 non-null   object 
 14  compound           2627 non-null   float64
 15  overall_sentiment  2627 non-null   object 
 16  Pos_minus_Neg      2627 non-n

,Unnamed: 0,CLOSE,Positive,Negative,Neutral,Total,summed_vader,sentiments,headlines,stocks,date,dayOfWeek,ave_vader,daily_sentiment,compound,overall_sentiment,Pos_minus_Neg,target,direction
2622,2755,25.81,321,72,335,728,146.5441,"['Positive', ' Neutral', ' Positive', ' Positi...",Shares of several apparel and retail companies...,"['AAN', ' AAN', ' AAPL', ' AAPL', ' AAPL', ' A...",2020-06-04,Thursday,0.201297,Positive,1.0000,Positive,249,24.52,0
2623,2756,24.52,720,85,354,1159,496.0841,"['Neutral', ' Positive', ' Positive', ' Neutra...","Stocks That Hit 52-Week Highs On Friday, Share...","['A', ' AAN', ' AAPL', ' AAPL', ' AAPL', ' AAP...",2020-06-05,Friday,0.428028,Positive,1.0000,Positive,635,25.81,1
2624,2757,25.81,425,68,415,908,206.5643,"['Positive', ' Neutral', ' Positive', ' Positi...",Shares of several retail and apparel companies...,"['AAN', ' AAN', ' AAPL', ' AAPL', ' AAPL', ' A...",2020-06-08,Monday,0.227494,Positive,1.0000,Positive,357,27.57,1
2625,2758,27.57,441,88,354,883,235.3178,"['Neutral', ' Neutral', ' Positive', ' Positiv...","Why Apple's Stock Is Trading Higher Today, App...","['AAPL', ' AAPL', ' AAPL', ' AAPL', ' AAPL', '...",2020-06-09,Tuesday,0.266498,Positive,1.0000,Positive,353,27.57,0
2626,2759,27.57,269,212,339,820,35.1903,"['Negative', ' Neutral', ' Negative', ' Positi...",Shares of retail & apparel companies are tradi...,"['AAN', ' AAN', ' AAP', ' AAPL', ' AAPL', ' AA...",2020-06-10,Wednesday,0.042915,Neutral,0.9996,Positive,57,40.79,1


In [55]:
data.values[0:4,17]

array([19.35, 19.16, 19.06, 18.13], dtype=object)

In [56]:
#90 percent training 10 percent testing
n_train = int(len(data) * 0.9)

# split data into train and test-sets
train, test = data[0:n_train], data[n_train:]
train.shape, test.shape

((2364, 19), (263, 19))

In [57]:
X_train = train.drop(columns=['CLOSE','direction', 'target'])
X_test  = test.drop(columns=['direction', 'target'])
y_train = train['target'].values
y_test = test['target'].values

In [58]:
categorical_col = ['sentiments','headlines','stocks','overall_sentiment','dayOfWeek']
numerical_col = ['Positive',
              'Negative', 'Neutral', 'Total','summed_vader','ave_vader','compound']

In [59]:
# pick transformers
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numerical_col)
   ,('categorical', categorical_transformer, categorical_col)
]) 

In [60]:
def preprocess(train,test):
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numerical_col),
            ('categorical', categorical_transformer, categorical_col)
        ])
    preprocessor = preprocessor.fit(train)
    # transform train
    #train = train.reshape(train.shape[0], train.shape[1])
    train_preprocessed = preprocessor.transform(train)
    # transform test
    #test = test.reshape(test.shape[0], test.shape[1])
    test_preprocessed = preprocessor.transform(test)
    return preprocessor, train_preprocessed, test_preprocessed
preprocessor, train_preprocessed, test_preprocessed = preprocess(X_train, X_test)

## GRU

In [61]:
# fit an GRU network to training data
def fit_gru(train, y_train, batch_size, nb_epoch, neurons):
    X, y = train , y_train
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(GRU(neurons,return_sequences = True,
              batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # Setting up an early stop
    earlystop = EarlyStopping(monitor='loss', min_delta=0.0001, patience=50,  verbose=1, mode='min')
    callbacks_list = [earlystop]
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False, callbacks=callbacks_list)
        model.reset_states()
    return model

In [65]:
#Batch Size: 1
#Epochs: 50
#Neurons: 500

gru_model = fit_gru(train_preprocessed, y_train, 1, 50, 500)
# forecast the entire training dataset to build up state for forecasting
#train_reshaped = train[:, :-1].reshape(len(train_scaled), 1, 11)
history = gru_model.predict(train_preprocessed, batch_size=1)

2364/2364 - 423s - loss: 12.7112 - 423s/epoch - 179ms/step
2364/2364 - 420s - loss: 10.7221 - 420s/epoch - 178ms/step
2364/2364 - 419s - loss: 8.7946 - 419s/epoch - 177ms/step
2364/2364 - 423s - loss: 6.4756 - 423s/epoch - 179ms/step
2364/2364 - 419s - loss: 4.8965 - 419s/epoch - 177ms/step
2364/2364 - 423s - loss: 5.0455 - 423s/epoch - 179ms/step
2364/2364 - 418s - loss: 6.4329 - 418s/epoch - 177ms/step
2364/2364 - 423s - loss: 6.4818 - 423s/epoch - 179ms/step
2364/2364 - 420s - loss: 6.2703 - 420s/epoch - 178ms/step
2364/2364 - 425s - loss: 6.7830 - 425s/epoch - 180ms/step
2364/2364 - 418s - loss: 7.9745 - 418s/epoch - 177ms/step
2364/2364 - 419s - loss: 7.2883 - 419s/epoch - 177ms/step
2364/2364 - 424s - loss: 7.9187 - 424s/epoch - 179ms/step
2364/2364 - 423s - loss: 5.0424 - 423s/epoch - 179ms/step
2364/2364 - 418s - loss: 6.4873 - 418s/epoch - 177ms/step
2364/2364 - 419s - loss: 6.5086 - 419s/epoch - 177ms/step
2364/2364 - 422s - loss: 5.7285 - 422s/epoch - 179ms/step
2364/2364 - 

ValueError: in user code:

    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\engine\training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\engine\training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\engine\training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\engine\training.py", line 2357, in predict_step
        return self(x, training=False)
    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\envs\snakes\lib\site-packages\keras\src\engine\input_

In [66]:
# make a one-step forecast
def forecast_gru(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0]

# inverse scaling for a forecasted value
def invert_scale(preprocessor, X, value):
    new_row = [x for x in X] + [value]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = preprocessor.inverse_transform(array)
    return inverted[0, -1]

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

In [67]:
# walk-forward validation on the test data
predictions = list()
for i in range(len(test_preprocessed)):
    # make one-step forecast
    X, y = test_preprocessed[i, 0:], y_test[i]
    yhat = forecast_gru(gru_model, 1, X)
    # invert scaling
    #yhat = invert_scale(scaler, X, yhat)
    # invert differencing
    #yhat = inverse_difference(data_values[:,0], yhat, len(test_scaled)+1-i)
    # store forecast
    predictions.append(yhat)
    expected = data_values[:,17][len(train) + i + 1]
    print('Day=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

1/1 [==============================] - 0s 322ms/step
Day=1, Predicted=12.282049, Expected=17.300000
1/1 [==============================] - 0s 23ms/step
Day=2, Predicted=11.508595, Expected=18.710000
1/1 [==============================] - 0s 22ms/step
Day=3, Predicted=11.652795, Expected=18.860000
1/1 [==============================] - 0s 24ms/step
Day=4, Predicted=17.939720, Expected=16.970000
1/1 [==============================] - 0s 23ms/step
Day=5, Predicted=9.394238, Expected=16.090000
1/1 [==============================] - 0s 23ms/step
Day=6, Predicted=13.301510, Expected=15.930000
1/1 [==============================] - 0s 24ms/step
Day=7, Predicted=14.729101, Expected=16.300000
1/1 [==============================] - 0s 24ms/step
Day=8, Predicted=11.348721, Expected=15.940000
1/1 [==============================] - 0s 23ms/step
Day=9, Predicted=11.253296, Expected=15.990000
1/1 [==============================] - 0s 23ms/step
Day=10, Predicted=10.308255, Expected=15.910000
1/1 [====

1/1 [==============================] - 0s 22ms/step
Day=83, Predicted=10.710511, Expected=15.960000
1/1 [==============================] - 0s 25ms/step
Day=84, Predicted=17.230171, Expected=16.070000
1/1 [==============================] - 0s 26ms/step
Day=85, Predicted=10.534915, Expected=17.220000
1/1 [==============================] - 0s 25ms/step
Day=86, Predicted=12.136114, Expected=16.240000
1/1 [==============================] - 0s 24ms/step
Day=87, Predicted=10.595096, Expected=18.560000
1/1 [==============================] - 0s 24ms/step
Day=88, Predicted=11.391753, Expected=20.560000
1/1 [==============================] - 0s 23ms/step
Day=89, Predicted=14.593601, Expected=19.120000
1/1 [==============================] - 0s 25ms/step
Day=90, Predicted=20.306683, Expected=17.040000
1/1 [==============================] - 0s 23ms/step
Day=91, Predicted=13.818939, Expected=17.860000
1/1 [==============================] - 0s 26ms/step
Day=92, Predicted=13.096017, Expected=20.280000


Day=164, Predicted=8.662236, Expected=12.910000
1/1 [==============================] - 0s 23ms/step
Day=165, Predicted=7.288910, Expected=12.980000
1/1 [==============================] - 0s 26ms/step
Day=166, Predicted=7.316138, Expected=14.560000
1/1 [==============================] - 0s 23ms/step
Day=167, Predicted=9.088654, Expected=18.230000
1/1 [==============================] - 0s 23ms/step
Day=168, Predicted=8.759913, Expected=16.280000
1/1 [==============================] - 0s 24ms/step
Day=169, Predicted=16.212460, Expected=16.390000
1/1 [==============================] - 0s 24ms/step
Day=170, Predicted=9.774652, Expected=15.490000
1/1 [==============================] - 0s 23ms/step
Day=171, Predicted=9.063837, Expected=18.840000
1/1 [==============================] - 0s 26ms/step
Day=172, Predicted=14.473553, Expected=17.970000
1/1 [==============================] - 0s 24ms/step
Day=173, Predicted=15.522701, Expected=16.050000
1/1 [==============================] - 0s 23ms/st

1/1 [==============================] - 0s 25ms/step
Day=245, Predicted=-18.025736, Expected=29.300000
1/1 [==============================] - 0s 25ms/step
Day=246, Predicted=-19.327469, Expected=30.530000
1/1 [==============================] - 0s 26ms/step
Day=247, Predicted=-20.917303, Expected=27.990000
1/1 [==============================] - 0s 25ms/step
Day=248, Predicted=-19.545713, Expected=29.530000
1/1 [==============================] - 0s 25ms/step
Day=249, Predicted=-19.857168, Expected=28.160000
1/1 [==============================] - 0s 24ms/step
Day=250, Predicted=-19.583578, Expected=28.010000
1/1 [==============================] - 0s 24ms/step
Day=251, Predicted=-18.095234, Expected=27.620000
1/1 [==============================] - 0s 26ms/step
Day=252, Predicted=-19.433102, Expected=28.590000
1/1 [==============================] - 0s 23ms/step
Day=253, Predicted=-21.429230, Expected=27.510000
1/1 [==============================] - 0s 23ms/step
Day=254, Predicted=-21.065306,

IndexError: index 2627 is out of bounds for axis 0 with size 2627